In [4]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import webcolors
from webcolors import name_to_rgb, rgb_to_name
matplotlib.use('TkAgg')

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

image_path = 'bus.jpg'

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

results = model(image_path)  # predict on an image

def find_closest_color_name(r, g, b):
    target_rgb = (r, g, b)

    def rgb_to_lab(rgb):
        # Convert RGB to CIE Lab color space
        r, g, b = [x / 255.0 for x in rgb]
        r = ((r + 0.055) / 1.055) ** 2.4 if (r > 0.04045) else (r / 12.92)
        g = ((g + 0.055) / 1.055) ** 2.4 if (g > 0.04045) else (g / 12.92)
        b = ((b + 0.055) / 1.055) ** 2.4 if (b > 0.04045) else (b / 12.92)

        r, g, b = [x * 100.0 for x in [r, g, b]]
        x = r * 0.4124564 + g * 0.3575761 + b * 0.1804375
        y = r * 0.2126729 + g * 0.7151522 + b * 0.0721750
        z = r * 0.0193339 + g * 0.1191920 + b * 0.9503041

        x = x / 95.047
        y = y / 100.000
        z = z / 108.883

        x = (x ** (1.0 / 3.0)) if (x > 0.008856) else (903.3 * x + 16.0) / 116.0
        y = (y ** (1.0 / 3.0)) if (y > 0.008856) else (903.3 * y + 16.0) / 116.0
        z = (z ** (1.0 / 3.0)) if (z > 0.008856) else (903.3 * z + 16.0) / 116.0

        l = max(0.0, (116.0 * y) - 16.0)
        a = (x - y) * 500.0
        b = (y - z) * 200.0

        return l, a, b

    def delta_e_cie76(color1, color2):
        # Calculate the color difference using CIE76 formula
        l1, a1, b1 = color1
        l2, a2, b2 = color2
        delta_l = l2 - l1
        delta_a = a2 - a1
        delta_b = b2 - b1
        return (delta_l ** 2 + delta_a ** 2 + delta_b ** 2) ** 0.5

    # Get the names and RGB values of all valid CSS3 colors
    all_color_data = [(name.lower(), webcolors.name_to_rgb(name)) for name in webcolors.CSS3_NAMES_TO_HEX]

    # Find the closest color name
    closest_color_name = min(all_color_data, key=lambda data: delta_e_cie76(rgb_to_lab(target_rgb), rgb_to_lab(data[1])))[0]

    return closest_color_name

for r in results:

    detection_count = r.boxes.shape[0]

    name = []
    for detection in range(detection_count):
        cls = int(r.boxes.cls[detection].item())
        name.append(r.names[cls])

    boxes = r.boxes.xyxy.numpy()
    color_list = []
    for i in range(len(boxes)):
        x1 = int(boxes[i][0])
        y1 = int(boxes[i][1])
        x2 = int(boxes[i][2])
        y2 = int(boxes[i][3])

        # get average color of bounding box
        average_color = np.mean(image[y1:y2, x1:x2], axis=(0, 1))
        # convert back to integer color
        average_color = [int(c) for c in average_color]
        # add average color to list
        color_list.append(average_color)

    # draw bounding boxes and labels of detections with color of average color of bounding box
    for i in range(len(boxes)):
        x1 = int(boxes[i][0])
        y1 = int(boxes[i][1])
        x2 = int(boxes[i][2])
        y2 = int(boxes[i][3])

        # draw bounding box with weight relative to image size
        cv2.rectangle(image, (x1, y1), (x2, y2), color_list[i], int(image.shape[0] / 100) + 1)

        # make label text size proportional to image size
        label_size = (image.shape[0] / 1000) * 1.5

        # draw background rectangle for label text make width proportional to name length
        (label_width, label_height), baseline = cv2.getTextSize(name[i], cv2.FONT_HERSHEY_SIMPLEX, label_size, 2)
        cv2.rectangle(image, (x1, y1), (x1 + label_width, y1 + int(label_size * 20)), color_list[i], -1)

        # draw label text
        cv2.putText(image, name[i], (x1, y1 + int(label_size * 20)), cv2.FONT_HERSHEY_SIMPLEX, label_size,
                    (255, 255, 255), 2)

        # in next row draw color name with background rectangle of color with function find_closest_color_name() add little gap between rows
        (label_width, label_height), baseline = cv2.getTextSize(find_closest_color_name(color_list[i][0], color_list[i][1], color_list[i][2]), cv2.FONT_HERSHEY_SIMPLEX, label_size, 2)
        cv2.rectangle(image, (x1, y1 + int(label_size * 20) + int(label_size * 20)), (x1 + label_width, y1 + int(label_size * 20) + int(label_size * 20) + int(label_size * 20)), color_list[i], -1)

        # draw color name
        cv2.putText(image, find_closest_color_name(color_list[i][0], color_list[i][1], color_list[i][2]), (x1, y1 + int(label_size * 20) + int(label_size * 20) + int(label_size * 20)), cv2.FONT_HERSHEY_SIMPLEX, label_size,
                    (255, 255, 255), 2)


# call imshow() using plt object
plt.imshow(image)

# display that image
plt.show()


image 1/1 d:\project\Clustering-Color\bus.jpg: 448x640 3 buss, 277.3ms
Speed: 7.0ms preprocess, 277.3ms inference, 5.0ms postprocess per image at shape (1, 3, 448, 640)
